## fixing first year stocks

## mporting libraries

In [16]:
import numpy as np
import pandas as pd
import datetime as dt
pd.set_option('display.max_rows',None)

##  reading file

In [17]:
master_df = pd.read_csv("C:\\keshav\\50stocks_performance\\masterFile.csv", parse_dates=['Date'],index_col='Date')
nifty_df = pd.read_csv("C:\\keshav\\historicalData2013-2023\\Nifty50.csv",usecols=['symbol','Outstanding shares'])
nifty_df.set_index('symbol',inplace=True)


#reading Beta1 sheet of excel file
beta_file = pd.read_excel("C:\\keshav\\historicalData2013-2023\\Beta_nifty.xlsx",sheet_name='Beta1', header=1, parse_dates=['Date'], index_col='Date')
beta_file = beta_file.round(1)

## inputs (dates)

In [18]:
start_date_input = pd.to_datetime("2012-03-31")
end_date_input = pd.to_datetime("2023-03-31")
filter_mode = "market_cap"  #"market_cap"
beta_value = 0.7


period = input(f"Please choose from [yearly,quarterly, monthly, custom]  : ")

if period == "custom":
    input_custom_days = int(input('Please choose custom days : '))

else:
    input_custom_days = 30 


### DATES WHICH IS PRESENT IN master_df

In [19]:
start_date = master_df[master_df.index <= start_date_input].index.max()
end_date = master_df[master_df.index <= end_date_input].index.max()


##   MAKING BINS --> annual, quarterly, monthly, custom

In [20]:
filtered_yearly_bins = []

# Iterate over each year from the start date until the end year
for year in range(start_date.year, end_date.year + 1):
    # Create a timestamp for March 31 of the current year
    target_date = pd.Timestamp(year=year, month=3, day=31)

    # Find the closest date before March 31 in the master dataframe index
    # closest_date = max(filter(lambda x: x <= target_date, master_df.index))
    closest_date = master_df.index[master_df.index <= target_date].max()

    filtered_yearly_bins.append(closest_date)


In [21]:
# %% Monthly filtered df
monthly_dates = pd.date_range(start=start_date, end=end_date, freq='M')
filtered_monthly_bins = []

# Iterate over each monthly date
for date in monthly_dates:
    closest_date = master_df.index[master_df.index <= date].max()
    filtered_monthly_bins.append(closest_date)

In [22]:
#%% Qurterly 
quarterly_dates = pd.date_range(start=start_date, end=end_date, freq='Q')
filtered_quarterly_bins = []

# Iterate over each quarterly date
for date in quarterly_dates:
    closest_date = master_df.index[master_df.index <= date].max()
    filtered_quarterly_bins.append(closest_date)


In [23]:
#%% custom dates 
custom_dates = pd.date_range(start=start_date, end=end_date, freq=f"{input_custom_days}D")
filtered_custom_bins = []

# Iterate over each custom date
for date in custom_dates:
    closest_date = master_df.index[master_df.index <= date].max()
    filtered_custom_bins.append(closest_date)

## making input period as type of bins

In [24]:
if period == 'yearly':
    filtered_dates = filtered_yearly_bins

elif period == 'monthly':
    filtered_dates = filtered_monthly_bins

elif period == 'quarterly':
    filtered_dates = filtered_quarterly_bins

elif period == 'custom':
    filtered_dates = filtered_custom_bins

In [32]:
filtered_dates

[Timestamp('2012-03-30 00:00:00'),
 Timestamp('2012-04-30 00:00:00'),
 Timestamp('2012-05-31 00:00:00'),
 Timestamp('2012-06-29 00:00:00'),
 Timestamp('2012-07-31 00:00:00'),
 Timestamp('2012-08-31 00:00:00'),
 Timestamp('2012-09-28 00:00:00'),
 Timestamp('2012-10-31 00:00:00'),
 Timestamp('2012-11-30 00:00:00'),
 Timestamp('2012-12-31 00:00:00'),
 Timestamp('2013-01-31 00:00:00'),
 Timestamp('2013-02-28 00:00:00'),
 Timestamp('2013-03-28 00:00:00'),
 Timestamp('2013-04-30 00:00:00'),
 Timestamp('2013-05-31 00:00:00'),
 Timestamp('2013-06-28 00:00:00'),
 Timestamp('2013-07-31 00:00:00'),
 Timestamp('2013-08-30 00:00:00'),
 Timestamp('2013-09-30 00:00:00'),
 Timestamp('2013-10-31 00:00:00'),
 Timestamp('2013-11-29 00:00:00'),
 Timestamp('2013-12-31 00:00:00'),
 Timestamp('2014-01-31 00:00:00'),
 Timestamp('2014-02-28 00:00:00'),
 Timestamp('2014-03-31 00:00:00'),
 Timestamp('2014-04-30 00:00:00'),
 Timestamp('2014-05-30 00:00:00'),
 Timestamp('2014-06-30 00:00:00'),
 Timestamp('2014-07-

In [51]:
# Filter only the first date of every year
filtered_dates2 = [date for i, date in enumerate(filtered_dates) if i == 0 or date.year != filtered_dates[i-1].year]
print(filtered_dates2)

[Timestamp('2012-03-30 00:00:00'), Timestamp('2013-01-31 00:00:00'), Timestamp('2014-01-31 00:00:00'), Timestamp('2015-01-30 00:00:00'), Timestamp('2016-01-29 00:00:00'), Timestamp('2017-01-31 00:00:00'), Timestamp('2018-01-31 00:00:00'), Timestamp('2019-01-31 00:00:00'), Timestamp('2020-01-31 00:00:00'), Timestamp('2021-01-29 00:00:00'), Timestamp('2022-01-31 00:00:00'), Timestamp('2023-01-31 00:00:00')]


In [42]:
filtered_dates
len(filtered_dates)

133

## want static top 10 only for every year not monthly

In [55]:
# betadates fitered
beta_file_filtered = beta_file.loc[filtered_dates]


#%% LOOP PART CAPTURING DETAILS AND INSERTING IN DICTIONARIES
main_dict = {}
nifty_dict = {}

for i in range(len(filtered_dates)):
    # print(filtered_dates[i])
    try:
            
        df = nifty_df.copy()
        for company in df.index:
            value = master_df.loc[filtered_dates[i],company]
            df.loc[company,'price'] = value
            
            #for next year
            value2 = master_df.loc[filtered_dates[i+1], company]
            df.loc[company,'next_year_price'] = value2

        if filtered_dates[i] in filtered_dates2:
            # if filter_mode == "beta":
            #     mask = beta_file_filtered.loc[filtered_dates[i]] < 0.7
            #     filtered_row = beta_file_filtered.loc[filtered_dates[i]][mask]
            #     # print(filtered_row)
            #     df = df[df.index.isin(filtered_row.index)]


            if filter_mode == 'market_cap':
                    df['market_cap'] = df['Outstanding shares']*df['price']
                    df = df.sort_values(by='market_cap', ascending=False).head(10)  #top 10
                    companies = df.index
                    print(companies)
                    print(df)
        
        else:
            df['return'] = df['next_year_price']/df['price']*100 -100
            
            print('hiii', df)
            # df = df['return'].mean()
            # main_dict[filtered_dates[i]] = df
            # print(df)



        # #also for nifty
        # #creating every time within loop
        # nifty_temp_df = pd.DataFrame(index=['Nifty'], columns=['price', 'next_year_price'])
        # value_nifty = master_df.loc[filtered_dates[i],'Nifty']
        # nifty_temp_df.loc['Nifty','price'] = value_nifty    

        # #for next year        
        # value2_nifty = master_df.loc[filtered_dates[i+1], 'Nifty']
        # nifty_temp_df.loc['Nifty','next_year_price'] = value2_nifty

        # nifty_temp_df['return'] = nifty_temp_df['next_year_price']/nifty_temp_df['price']*100 -100
        # nifty_temp_df = nifty_temp_df['return'].values[0]
        # nifty_dict[filtered_dates[i]] = nifty_temp_df

    except:
        pass



Index(['RELIANCE', 'BHARTIARTL', 'TCS', 'SBIN', 'ITC', 'ONGC', 'HDFCBANK',
       'INFY', 'HDFC', 'ICICIBANK'],
      dtype='object', name='symbol')
            Outstanding shares       price  next_year_price    market_cap
symbol                                                                   
RELIANCE          6.729321e+09  342.146423       339.639343  2.302413e+12
BHARTIARTL        5.925344e+09  293.171906       269.398773  1.737144e+12
TCS               3.639312e+09  463.906189       494.785339  1.688300e+12
SBIN              8.884738e+09  183.963226       187.626968  1.634465e+12
ITC               1.247841e+10  112.173622       121.368988  1.399749e+12
ONGC              1.246902e+10  108.092209       108.776924  1.347804e+12
HDFCBANK          5.561381e+09  237.801544       248.162628  1.322505e+12
INFY              4.113267e+09  271.310150       233.050537  1.115971e+12
HDFC              1.822213e+09  562.555847       562.806702  1.025097e+12
ICICIBANK         6.971011e+09  140.2

##  Experimental monthly all

## final result printing

In [26]:
returns_df1 = pd.DataFrame.from_dict(main_dict, orient='index', columns=['top10_stocks_avg_returns'])
returns_df2 = pd.DataFrame.from_dict(nifty_dict, orient='index', columns=['nifty_returns'])
final_df = pd.merge(returns_df1,returns_df2, left_index=True, right_index=True)
final_df['diff'] = final_df['top10_stocks_avg_returns'] - final_df['nifty_returns']
final_df = final_df.round(1)
print(final_df)


            top10_stocks_avg_returns  nifty_returns  diff
2012-03-30                      -0.2           -0.9   0.7
2012-04-30                      -6.7           -6.2  -0.5
2012-05-31                       7.3            7.2   0.1
2012-06-29                      -0.5           -0.9   0.5
2012-07-31                      -1.5            0.6  -2.1
2012-08-31                       7.3            8.5  -1.1
2012-09-28                      -0.8           -1.5   0.6
2012-10-31                       5.7            4.6   1.1
2012-11-30                       0.0            0.4  -0.4
2012-12-31                       4.1            2.2   1.9
2013-01-31                      -4.2           -5.7   1.5
2013-02-28                       0.3           -0.2   0.5
2013-03-28                       2.0            4.4  -2.4
2013-04-30                       1.5            0.9   0.5
2013-05-31                      -1.6           -2.4   0.8
2013-06-28                      -0.8           -1.7   1.0
2013-07-31    

In [118]:
# final_df.to_csv('monthly.csv')